# Creates the separate and overlapping scatter plots

### Load the libraries and data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.lines as mlines
from scipy.stats import gaussian_kde

In [ ]:
# Load the data
DATA_EXCEL_PATH = ""
SHEET_NAME = ""
df = pd.read_excel(DATA_EXCEL_PATH, sheet_name=SHEET_NAME)

### Process the data

In [ ]:
# Get the appropriate columns
lt_x = []
lt_y = []
ht_x = []
ht_y = []
liq_x = []
liq_y = []
for c in df.columns:
    if 'LT' in c:
        series = df.loc[df[c] > 0, ['RT1', 'RT2', c]]
        lt_x.extend(series['RT1'])
        lt_y.extend(series['RT2'])
    elif 'HT' in c:
        series = df.loc[df[c] > 0, ['RT1', 'RT2', c]]
        ht_x.extend(series['RT1'])
        ht_y.extend(series['RT2'])
    elif 'LIQ' in c:
        series = df.loc[df[c] > 0, ['RT1', 'RT2', c]]
        liq_x.extend(series['RT1'])
        liq_y.extend(series['RT2'])
    else:
        print('Skipping', c)

In [ ]:
# Perform the density estimation
lt_xy = np.vstack([lt_x, lt_y])
lt_z = gaussian_kde(lt_xy)(lt_xy)
print("Done with LT")
ht_xy = np.vstack([ht_x, ht_y])
ht_z = gaussian_kde(ht_xy)(ht_xy)
print("Done with HT")
liq_xy = np.vstack([liq_x, liq_y])
liq_z = gaussian_kde(liq_xy)(liq_xy)

### Create the plots

In [ ]:
# Create the three separate plots
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey='row', figsize=(15, 5))
ax1.set_title('LIQ')
ax1.set_xlabel('RT1 (s)')
ax1.set_ylabel('RT2 (s)')
ax1.scatter(liq_x, liq_y, c=liq_z**0.5, s=100, cmap='Blues')
ax2.set_title('$\mathregular{T_H}$')
ax2.set_xlabel('RT1 (s)')
ax2.scatter(ht_x, ht_y, c=ht_z**0.5, s=100, cmap='Reds')
ax3.set_title('$\mathregular{T_L}$')
ax3.set_xlabel('RT1 (s)')
ax3.scatter(lt_x, lt_y, c=lt_z**0.5, s=100, cmap='Greens')
fig.savefig('scatter_3.png', dpi=300, bbox_inches='tight')

In [ ]:
# Create the overlay plot
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(liq_x, liq_y, c=liq_z**0.5, s=100, alpha=0.05, cmap='Blues')
ax.scatter(ht_x, ht_y, c=ht_z**0.5, s=100, alpha=0.05, cmap='Reds')
ax.scatter(lt_x, lt_y, c=lt_z**0.5, s=100, alpha=0.05, cmap='Greens')
lt_label = mlines.Line2D([], [], color='green', marker='o', alpha=0.5, linestyle='None',
                          markersize=10, label='Low temperature')
ht_label = mlines.Line2D([], [], color='red', marker='o', alpha=0.5, linestyle='None',
                          markersize=10, label='High temperature')
liq_label = mlines.Line2D([], [], color='blue', marker='o', alpha=0.5, linestyle='None',
                          markersize=10, label='Liquid extraction')
plt.xlabel('RT1 (s)')
plt.ylabel('RT2 (s)')
plt.title('Overlay')
plt.legend(handles=[lt_label, ht_label, liq_label])
plt.savefig(r'scatter_overlay_full_labels.png', dpi=300)